## **Turi Text Classifier**

Welcome to the text classification model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

1. Load labeled text training data.
2. Build a sentiment classification model.
3. Convert the model to CoreML format and save it to the Skafos framework.

The example is based on [Turi Create's Text Classifier](https://apple.github.io/turicreate/docs/userguide/text_classifier/).

---

Execute each cell one-by-one, by selecting the cell and do one of the following:
-  Clicking the "play" button at the top of this frame.
-  Typing 'Control + Enter' or 'Shift + Enter'.



In [ ]:
# Import necessary libraries
from skafossdk import *

import utilities.save_models as sm
from utilities.load_turicreate import *

In [ ]:
tc = install_turicreate(timeout=500, retries=2) # check to make sure turicreate is installed
ska = Skafos() # initialize Skafos

### 1. **Load the data**
The training data for this example is Yelp review data, paired with sentiment scores. The data is randomly split into train and test sets, where 80% of the data is used for training, and 20% is used for model evaluation.

In [ ]:
# Load data from Turi Create's website
data = tc.SFrame('https://static.turi.com/datasets/regression/yelp-data.csv')

# Make a train-test split
train_data, test_data = data.random_split(0.8)

In [ ]:
# Take a look at the data
train_data.head(5)

### 2. **Build the model**
We use the `tc.text_classifier.create` function and specify the data, target variable, features, and a few other arguments needed to properly train the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/api/generated/turicreate.text_classifier.create.html#turicreate.text_classifier.create).

In [ ]:
# Train a sentiment classification model, this takes approximately 15 minutes using CPU
model = tc.text_classifier.create(
    dataset=train_data,
    target='stars',
    features=['text'],
    drop_stop_words=True,
    max_iterations=10
)

In [ ]:
# Now that the model is trained, we can evaluate against a test set
test_predictions = model.predict(test_data)
accuracy = tc.evaluation.accuracy(test_data['stars'], predictions)
ska.log(f'Sentiment model has a testing accuracy of {accuracy*100} % !', labels=['model_evaluation'])

### 3. **Save the model**
Once your model has been created, it must be saved to thr Skafos framework via the code below. This will trigger a push to your mobile app.

In [ ]:
# Export the model to CoreML
coreml_model_name = "text_classifier.mlmodel"
res = model.export_coreml(coreml_model_name)

# Compress the model
compressed_model_name, compressed_model = sm.compress_model(coreml_model_name)

# Save to Skafos
sm.skafos_save_model(
    skafos=ska,
    model_name=compressed_model_name,
    compressed_model=compressed_model,
    permissions='public'
)